# Лабораторная работа №3
### Задание
Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Выполнение работы:
### Шаг 1. Загрузка датасета

In [5]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd


df = pd.read_csv("./base_v3.csv")

y = df["is_fraud"]
X = df.drop("is_fraud", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Шаг 2. Линейная регрессия
Линейная регрессия - это метод, используемый для предсказания значения переменной на основе линейной комбинации её признаков. 

In [6]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_accuracy = linear_model.score(X_test, y_test)

## Логистическая регресия

## Описание гиперпараметров логистической регрессии

### C (Обратная сила регуляризации):

Описание: C - это параметр регуляризации, который контролирует обратную силу регуляризации. Меньшие значения C увеличивают силу регуляризации. В контексте логистической регрессии, C определяет, насколько модель будет подстраиваться под тренировочные данные, минимизируя ошибку, и при этом учитывая сложность модели. Большие значения C могут привести к переобучению, если данных мало.

### Penalty (Тип регуляризации):

Описание: Этот параметр определяет вид регуляризации, которую следует применять. L1-регуляризация (Lasso) добавляет штраф для суммы абсолютных значений коэффициентов, что может привести к отбору признаков. L2-регуляризация (Ridge) добавляет штраф для суммы квадратов коэффициентов, что может предотвратить большие значения коэффициентов.

### Solver (Алгоритм оптимизации):

Описание: Solver - это алгоритм оптимизации, используемый для решения задачи оптимизации при подгонке модели. Различные алгоритмы могут подходить для разных задач и данных.
'liblinear' подходит для небольших наборов данных, поддерживает L1 и L2 регуляризацию.
'saga' поддерживает большие наборы данных и может обрабатывать как L1, так и L2 регуляризацию.
'lbfgs' также подходит для маленьких и средних по размеру наборов данных.

## Перескрестная проверка гиперпараметров

In [7]:
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga', 'lbfgs']
}

logistic = LogisticRegression(max_iter=1000)
grid = GridSearchCV(logistic, param_grid, refit=True,n_jobs=-1)
grid.fit(X_train, y_train)

C:\Users\Danil\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Danil\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Danil\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Danil\AppData\Roaming\Python\Python311\site-

GridSearchCV(estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga', 'lbfgs']})

Шаг 6. Вывод результатов

In [8]:
# Оценка производительности модели логистической регрессии на тестовом наборе
best_logistic = grid.best_estimator_
logistic_accuracy = best_logistic.score(X_test, y_test)
print(f"Лучшие гиперпараметры логистической регрессии: {grid.best_params_}")
print(f"Точность модели логистической регрессии: {logistic_accuracy}")
print(f"Точность модели линейной регрессии: {linear_accuracy}")

# Предсказания на тестовой выборке с использованием лучшей модели
logistic_predictions = best_logistic.predict(X_test)
linear_prediction = linear_model.predict(X_test)
print(classification_report(y_test, logistic_predictions))

# Отчет по классификации
print("\nОтчет по классификации для линейной регрессии:")
# Метрики регрессии
mse = mean_squared_error(y_test, linear_prediction)
r2 = r2_score(y_test, linear_prediction)

print("Средняя квадратичная ошибка:", mse)
print("Коэффициент детерминации (R^2):", r2)


Лучшие гиперпараметры логистической регрессии: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Точность модели логистической регрессии: 0.9154166666666667
Точность модели линейной регрессии: 0.43282696948663024
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      2038
           1       0.88      0.51      0.65       362

    accuracy                           0.92      2400
   macro avg       0.90      0.75      0.80      2400
weighted avg       0.91      0.92      0.91      2400


Отчет по классификации для линейной регрессии:
Средняя квадратичная ошибка: 0.0726450184547607
Коэффициент детерминации (R^2): 0.43282696948663024
